<a href="https://colab.research.google.com/github/nerobite/neural_networks/blob/main/%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Реализуйте обучение нейронной сети из одного нейрона для задачи предсказания стоимости квартир boston house prices или california housing prices с использованием pytorch.

Инструкция к выполнению задания:

1. Загрузите и подготовьте данные

2. Разделите данные на test и train

3. Создать модель (объект) класса. Для создания объекта можно использовать класс Sequential

4. Обучите модель на train данных

5. Проверьте качество модели на тестовых данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
import torch
import random
import torch.nn as nn
import numpy as np

In [36]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv('housing.csv', header=None, delimiter=r"\s+", names=column_names)
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [37]:
y_pd = df['MEDV']
X_pd = df.drop(['MEDV'], axis=1)

In [38]:
scaler = StandardScaler()
X_sc = pd.DataFrame(scaler.fit_transform(X_pd), columns=X_pd.columns)
X_np = X_sc.to_numpy()
y_np = y_pd.to_numpy()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

In [40]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

In [41]:
y_train.shape = (y_train.shape[0], 1)
y_test.shape = (y_test.shape[0], 1)

In [42]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404, 1), (102, 1))

In [43]:
# Определение модели
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    # пропускаем данные через слои нс
    # вызывается автоматически в model(X_train)
    def forward(self, x):
        out = self.linear(x)
        return out

In [44]:
# Преобразование данных в тензоры PyTorch
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

In [45]:
# Определение гиперпараметров
input_size = X_train.shape[1]  # Размерность входных признаков
output_size = 1          # Размерность выхода (одна целевая переменная)
learning_rate = 0.01     # Скорость обучения
num_epochs = 100         # Количество эпох

In [46]:
# Создание экземпляра модели
model = LinearRegression(input_size, output_size)

In [47]:
# Определение функции потерь и оптимизатора
# определяет функцию потерь (MSELoss)
criterion = nn.MSELoss()
# создает оптимизатор (SGD), который будет обновлять параметры модели
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [48]:
# Цикл обучения
for epoch in range(num_epochs):

    # Forward pass (прямое распространение)

    # получает предсказанные значения
    outputs = model(X_train)
    # вычисляет значение функции потерь между outputs и y_train
    loss = criterion(outputs, y_train)

    # Backward pass и оптимизация

    # обнуляет градиенты параметров модели перед backward.
    optimizer.zero_grad()
    # вычисляет градиенты функции потерь по параметрам модели.
    loss.backward()
    # обновляет параметры модели, используя градиенты
    optimizer.step()

    # Вывод промежуточной информации
    if (epoch + 1) % 10 == 0:
        print(f'Эпоха [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Эпоха [10/100], Loss: 402.5630
Эпоха [20/100], Loss: 270.9077
Эпоха [30/100], Loss: 186.6302
Эпоха [40/100], Loss: 131.4475
Эпоха [50/100], Loss: 95.0954
Эпоха [60/100], Loss: 71.0552
Эпоха [70/100], Loss: 55.1021
Эпоха [80/100], Loss: 44.4802
Эпоха [90/100], Loss: 37.3840
Эпоха [100/100], Loss: 32.6263


In [49]:
# Вывод результатов
print('Параметры модели:')
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f'{name}: {param.data.numpy()}')

Параметры модели:
linear.weight: [[-0.5649023   0.11124187 -0.360196    0.85650885 -0.46018913  3.4984455
   0.04838057 -1.2346257   0.26050514 -0.46186244 -1.9926511   0.9203214
  -3.1143234 ]]
linear.bias: [19.458366]


In [50]:
with torch.no_grad():
    predicted = model(X_test).detach().numpy()

In [51]:
def calculate_mse(predicted, actual):

    assert len(predicted) == len(actual), "Длины массивов должны быть одинаковыми."

    # Вычисление суммы квадратов разностей
    squared_errors = np.square(predicted - actual)
    sum_squared_errors = np.sum(squared_errors)

    # Вычисление MSE
    mse = sum_squared_errors / len(predicted)

    return mse

In [52]:
mse = calculate_mse(predicted, y_test.numpy())
print("TEST MSE:", mse)

TEST MSE: 38.63050752527573


In [ ]:
#так себе результат на одном нейроне...

In [53]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(input_size, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 16),
    torch.nn.BatchNorm1d(16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, output_size)
)

In [58]:
# Определение гиперпараметров
input_size = X_train.shape[1]  # Размерность входных признаков
output_size = 1          # Размерность выхода (одна целевая переменная)
learning_rate = 0.01     # Скорость обучения
num_epochs = 100         # Количество эпох

In [61]:
# Определение функции потерь и оптимизатора
# определяет функцию потерь (MSELoss)
criterion = torch.nn.MSELoss()
# создает оптимизатор (SGD), который будет обновлять параметры модели
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [62]:
# Цикл обучения
for epoch in range(num_epochs):

    # Forward pass (прямое распространение)

    # получает предсказанные значения
    outputs = model(X_train)
    # вычисляет значение функции потерь между outputs и y_train
    loss = criterion(outputs, y_train)

    # Backward pass и оптимизация

    # обнуляет градиенты параметров модели перед backward.
    optimizer.zero_grad()
    # вычисляет градиенты функции потерь по параметрам модели.
    loss.backward()
    # обновляет параметры модели, используя градиенты
    optimizer.step()

    # Вывод промежуточной информации
    if (epoch + 1) % 10 == 0:
        print(f'Эпоха [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Эпоха [10/100], Loss: 13.8855
Эпоха [20/100], Loss: 7.4946
Эпоха [30/100], Loss: 6.2476
Эпоха [40/100], Loss: 4.4989
Эпоха [50/100], Loss: 4.7897
Эпоха [60/100], Loss: 3.7416
Эпоха [70/100], Loss: 3.6307
Эпоха [80/100], Loss: 3.2316
Эпоха [90/100], Loss: 3.5734
Эпоха [100/100], Loss: 2.7610


In [ ]:
#вот так уже получше)